In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['ml_outcomes'] = df['ml_outcomes_div'].round(2) #+ df['R']*0.13



In [3]:
len(df)

100000

In [4]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [5]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','ml_outcomes_black','ml_outcomes_white','ml_outcomes_decision']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,ml_outcomes_black,ml_outcomes_white,ml_outcomes_decision
87320,0,15,0.04,9,15,-3.823794,-3.135931,0
39625,0,16,0.04,16,16,-3.272046,-3.057110,0
23572,0,15,0.04,14,15,-3.429688,-3.135931,0
24019,0,13,0.04,10,13,-3.744973,-3.293573,0
18378,0,13,0.04,11,13,-3.666152,-3.293573,0
...,...,...,...,...,...,...,...,...
40292,1,98,1.21,98,114,3.191292,4.667368,1
28577,1,99,1.21,99,114,3.270114,4.667368,1
97213,1,99,1.21,99,115,3.270114,4.746189,1
68116,1,102,1.22,102,118,3.506577,4.982653,1


In [6]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','ml_outcomes']].groupby(['R','T']).sum().reset_index()

In [7]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes']].groupby(['R','T']).count().reset_index()
df_count.columns = ['R','T','Count']
df_count['N'] = df_count['Count']

In [8]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [9]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,ml_outcomes,N
2,192,2,0,16,0.04,1
3,191,3,0,17,0.05,1
0,190,0,0,13,0.12,3
1,189,1,0,15,0.12,3
104,188,104,1,11,0.27,1
...,...,...,...,...,...,...
44,4,44,0,58,1083.60,1935
47,3,47,0,61,1094.30,1765
46,2,46,0,60,1111.20,1852
45,1,45,0,59,1132.16,1952


### Setup optimization problem 

In [10]:
from ortools.linear_solver import pywraplp


In [11]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [12]:
dff

,level_0,index,R,T,ml_outcomes,N
0,190,0,0,13,0.12,3
1,189,1,0,15,0.12,3
2,192,2,0,16,0.04,1
3,191,3,0,17,0.05,1
4,184,4,0,18,0.60,12
...,...,...,...,...,...,...
188,173,188,1,96,1.20,1
189,172,189,1,98,1.21,1
190,162,190,1,99,2.42,2
191,171,191,1,102,1.22,1


In [13]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [14]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [15]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [16]:
# Now we have one constraint
solver.NumConstraints()

1

## Add CF Fair constraints

In [17]:
from collections import Counter

def convertListToProb(raw_list):
    counts = dict(Counter(raw_list))
    probs = {}
    for test_score in counts:
        probs[test_score]  = counts[test_score]/float(len(raw_list))
    return [(probs[t], t) for t in probs]

In [18]:
T_blacks_list = df[df['R']==0][['T','T_black_star']].groupby('T')['T_black_star'].apply(list).reset_index(name='T_blacks')


In [19]:
T_blacks_list['probs'] = T_blacks_list['T_blacks'].apply(convertListToProb)


In [20]:
T_blacks_list

,T,T_blacks,probs
0,13,"[12, 13, 12]","[(0.6666666666666666, 12), (0.3333333333333333..."
1,15,"[16, 15, 13]","[(0.3333333333333333, 16), (0.3333333333333333..."
2,16,[16],"[(1.0, 16)]"
3,17,[17],"[(1.0, 17)]"
4,18,"[16, 18, 16, 16, 17, 16, 17, 17, 17, 16, 17, 17]","[(0.4166666666666667, 16), (0.0833333333333333..."
...,...,...,...
97,112,[103],"[(1.0, 103)]"
98,113,"[103, 105]","[(0.5, 103), (0.5, 105)]"
99,115,[106],"[(1.0, 106)]"
100,123,"[115, 113]","[(0.5, 115), (0.5, 113)]"


In [21]:
didntexist = 0
exists = 0
for ix, row in T_blacks_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    White_T = row['T']
    Blacks_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(0.0, White_T)], -1.0)
    for prob in Blacks_Ts:
        if (1.0, prob[1]) not in vars_cache:
            vars_cache[(1.0, prob[1])] = solver.NumVar(0.0, 1.0, str((1.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
        cf_fair_stratum.SetCoefficient(vars_cache[(1.0, prob[1])], prob[0])
    

In [22]:
didntexist,exists

(10, 548)

In [23]:
T_whites_list = df[df['R']==1][['T','T_white_star']].groupby('T')['T_white_star'].apply(list).reset_index(name='T_whites')

In [24]:
T_whites_list['probs'] = T_whites_list['T_whites'].apply(convertListToProb)


In [25]:
didntexist = 0
exists = 0

for ix, row in T_whites_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    Black_T = row['T']
    White_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(1.0, Black_T)], -1.0)
    for prob in White_Ts:
        if (0.0, prob[1]) not in vars_cache:
            vars_cache[(0.0, prob[1])] = solver.NumVar(0.0, 1.0, str((0.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
            
        cf_fair_stratum.SetCoefficient(vars_cache[(0.0, prob[1])], prob[0])
    

In [26]:
cf_fair_stratum.basis_status

<bound method Constraint.basis_status of <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fc45806bcf0> >>

In [27]:
solver.NumConstraints()

194

## Solve linear program

In [28]:
solver.ABNORMAL

4

In [29]:
status = solver.Solve()


In [30]:
status

0

In [31]:
solver.OPTIMAL

0

In [32]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [33]:
df_decisions

,row_id,decision
0,0,0.111057
1,1,0.435686
2,2,0.499757
3,3,0.499757
4,4,0.499757
...,...,...
188,188,0.897009
189,189,1.000000
190,190,1.000000
191,191,1.000000


In [34]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
42,0,42,0,56,1134.12,2181,42,0.499757
45,1,45,0,59,1132.16,1952,45,0.499757
46,2,46,0,60,1111.20,1852,46,0.499757
47,3,47,0,61,1094.30,1765,47,0.499757
44,4,44,0,58,1083.60,1935,44,0.499757
...,...,...,...,...,...,...,...,...
104,188,104,1,11,0.27,1,104,0.000000
1,189,1,0,15,0.12,3,1,0.435686
0,190,0,0,13,0.12,3,0,0.111057
3,191,3,0,17,0.05,1,3,0.499757


In [35]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
42,0,42,0,56,1134.12,2181,42,0.499757
45,1,45,0,59,1132.16,1952,45,0.499757
46,2,46,0,60,1111.20,1852,46,0.499757
47,3,47,0,61,1094.30,1765,47,0.499757
44,4,44,0,58,1083.60,1935,44,0.499757
...,...,...,...,...,...,...,...,...
104,188,104,1,11,0.27,1,104,0.000000
1,189,1,0,15,0.12,3,1,0.435686
0,190,0,0,13,0.12,3,0,0.111057
3,191,3,0,17,0.05,1,3,0.499757


In [36]:
xxx.sort_values(by='decision',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
191,171,191,1,102,1.22,1,191,1.000000
96,156,96,0,111,3.96,4,96,1.000000
100,164,100,0,123,2.00,2,100,1.000000
95,166,95,0,109,1.98,2,95,1.000000
98,167,98,0,113,1.98,2,98,1.000000
...,...,...,...,...,...,...,...,...
106,185,106,1,14,0.56,2,106,0.305407
0,190,0,0,13,0.12,3,0,0.111057
102,186,102,1,7,0.27,1,102,0.000000
103,187,103,1,10,0.27,1,103,0.000000


In [37]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)

In [38]:
admit_decisions = df.merge(xxx,how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_BLACK_POLICY = (admit_decisions['R_y'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R_y'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()


In [39]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Path-Specific Fairness',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()


In [40]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.5000000000000002